In [1]:
#OPTIMUS PORTFOLIO
#PROVIDING A PORTFOLIO SUGGESTION

In [2]:
#IMPORT DEPENDENCIES
import pandas as pd
import yfinance as yf
import datetime
import numpy as np

In [3]:
#APIs
# Define the list of stock ticker symbols
tickers = ['AGG', 'BNDX', 'SPY', 'IVW', 'IVE', 'BTC-USD', 'ETH-USD']

# Calculate the start date (three years ago from today)
start_date = datetime.datetime.now() - datetime.timedelta(days=1095)

# Initialize an empty dataframe to store the Close columns for each ticker
new_df = pd.DataFrame()

# Initialize an empty list to store the column names of Close prices for each ticker
ticker_closes = []

In [4]:
#Past 5 years of data into data frames
# Iterate over each ticker symbol
for ticker in tickers:
    # Fetch historical stock price data for the current ticker
    stock_data = yf.download(ticker, start=start_date, end=datetime.datetime.now())
    
    # Extract the 'Close' column and rename it for the current ticker
    close_column = stock_data[['Close']].rename(columns={'Close': f'{ticker}_Close'})
    
    # Compute the daily returns and insert them to the left of the corresponding ticker's Close column
    daily_returns = close_column.pct_change().rename(columns={f'{ticker}_Close': f'{ticker}_Daily_Return'})
    new_df = pd.concat([daily_returns, new_df], axis=1)
    
    # Append the Close column name for the current ticker to the ticker_closes list
    ticker_closes.append(f'{ticker}_Close')

# Drop rows containing NaNs from the new_df dataframe
new_df.dropna(inplace=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
## Feel free to adjust or send feedback if the code could be better or be more precise to what is needed:

span = len(new_df)  # Adjust the span for three years of historical data

# Calculate the annualized exponentially weighted mean return for each asset
annualized_mean_return = new_df.ewm(span=span).mean().iloc[-1] * 252

# Calculate the annualized exponentially weighted standard deviation (risk) for each asset
annualized_std_dev = new_df.ewm(span=span).std().iloc[-1] * np.sqrt(252)

# Display annualized risk and return for each asset
for ticker in tickers:
    print(f'{ticker}:')
    print(f'Annualized Mean Return: {annualized_mean_return[f"{ticker}_Daily_Return"]:.2%}')
    print(f'Annualized Risk (Standard Deviation): {annualized_std_dev[f"{ticker}_Daily_Return"]:.2%}')
    print()

print(new_df)

AGG:
Annualized Mean Return: -5.88%
Annualized Risk (Standard Deviation): 6.82%

BNDX:
Annualized Mean Return: -5.22%
Annualized Risk (Standard Deviation): 6.03%

SPY:
Annualized Mean Return: 7.51%
Annualized Risk (Standard Deviation): 17.44%

IVW:
Annualized Mean Return: 7.38%
Annualized Risk (Standard Deviation): 21.37%

IVE:
Annualized Mean Return: 7.51%
Annualized Risk (Standard Deviation): 15.20%

BTC-USD:
Annualized Mean Return: -1.96%
Annualized Risk (Standard Deviation): 56.43%

ETH-USD:
Annualized Mean Return: -4.52%
Annualized Risk (Standard Deviation): 70.91%

            ETH-USD_Daily_Return  BTC-USD_Daily_Return  IVE_Daily_Return  \
Date                                                                       
2021-05-04             -0.051720             -0.067600          0.001695   
2021-05-05              0.082724              0.076696          0.002640   
2021-05-06             -0.009056             -0.017893          0.008911   
2021-05-07             -0.001762          

In [6]:
#weights for portfolio - solver code targeting 5 portfolios


In [7]:
#pull weights into data frame


In [8]:
#visualization on each portfolio


In [9]:
#ask user for inputs


In [10]:
#return visualization and simulation monte carlo of closest portfolio to user input above

In [11]:
# print a congrats you are on your way to $$$
